# First Homework: Linear and Discrete Models

**Name:** Marina Rodríguez Hernández

**NIA:** 100429893

## Exercises a and b: Formulate problem and implement model

For this first project, I was looking for something that could be practical and could help people. 
One day, I was talking with my parents and I asked my dad how his day was going. He told me he had spent all morning trying to figure out how to distribute his patients in order to be able to operate on the maximum number of people in a week, so they do not need to wait a long period of time until they can get surgery. 
This made me realize that there is always a giant waiting list for surgical procedures in public hospitals, so I thought that maybe finding a way to optimize the order of procedures depending on the time it takes to operate, could make the waiting time much lower for people needing these operations, and for surgeons because they wouldn't need to manage this.

Problem formulation:

· **Decision variables:** different procedures

· **Objective:** maximize the hours a doctor is doing surgery so more people can have their operation

· **Constraints:** maximum hours per week, minimum hours per week, maximum number of patients per procedure

In [23]:
from pyomo.environ import *

import pandas as pd

In order to do this, I will be using a data set that shows:

· An identifier for each patient

· The diagnostic they have

· The procedure they need

· The hours it takes to operate each procedure

· The doctor assigned 

· The number of people that need the same operation. 

In total it has 6 columns and 85 rows (85 patients waiting to have surgery).

In [24]:
df = pd.read_excel('Waiting2.xlsx')  
print(df)

    Patient                        Diagnostic                     Procedure  \
0        83  Listesis L5-S1+ discopatia L4-l5             Abordaje anterior   
1        91                               HB4             Abordaje anterior   
2        35                 Adenoma hipofisis          Abordaje endoscópico   
3        61                             225.0          Abordaje endoscópico   
4        62                 Adenoma hipofisis          Abordaje endoscópico   
..      ...                               ...                           ...   
80       88                            722.10                        P80.51   
81       25             Sdme postlaminectomía            Retirada tornillos   
82        4               Fract periprotésica           Retirada tornillos    
83        5        Estenosis de canal  lumbar           Retirada tornillos    
84       90           Espasticidad/paraplejia  Terapia intratecal Baclofeno   

    Group  médico  Num of Procedures  
0     3.5   

First of all, I imported the dataset calling it df. Then I created different lists and dictionaries.

· proc is a list with all the different procedures, these will be my **variables**

· hours is a dictionary with the different procedures and the time it takes to operate them

· numProc is a dictionary with the different procedures and how many people are waiting for the same operation

· patient is a dictionary with the different procedures and the patients that need that procedure

In [25]:
# Import data using pandas

proc = list(set(df['Procedure'])) 

hours = df.set_index(['Procedure'])['Group'].to_dict()

numProc = df.set_index(['Procedure'])['Num of Procedures'].to_dict()

patient = df.set_index(['Procedure'])['Patient'].to_dict()

Then, we need to define the model, the variables, and the objective.
As I said before, the variables are the different procedures, and I will try to calculate how many should a surgeon operate in a week. The **objective** consists in maximizing the hours a doctor is doing surgery, in order to be able to have as many surgeries as possible.

In [26]:
## DEFINE MODEL ##
model = ConcreteModel()

#Define the variables
model.x=Var(proc, domain=NonNegativeIntegers)


#Define the objective: maximize the hours a doctor spends operating
def obj_expression(model):
    return sum(model.x[i] * hours[i] for i in proc)

model.objective=Objective(rule=obj_expression, sense=maximize)


After this, I need to define the **constraints**. First, I defined the maximum (40 hours) and the minimum (1 hour) number of hours a doctor can work in a week.

In [27]:
#constraints
def upper_limit(model):
    suma= sum(model.x[i] * hours[i] for i in proc) 
    return suma <= 40

def lower_limit(model):
    suma= sum(model.x[i] * hours[i] for i in proc) 
    return suma >= 1 

model.constraint1=Constraint(rule=upper_limit)
model.constraint2=Constraint(rule=lower_limit)

Then, since there are some patients needing the same procedures, we could have the same procedure multiple times. In the numProc variable, you can see how many times a doctor needs to follow the same procedure in different people.

In [28]:
print(numProc)

{'Abordaje anterior': 2, 'Abordaje endoscópico': 4, 'Artrodesis cervical': 4, 'Artrodesis lumbar': 4, 'Craneoplastia': 1, 'Craneotomía': 5, 'Descompresión': 2, 'Disc c3-c4 yc 4-c5 + PEEK': 1, 'Disc caja': 2, 'disc caja placa': 2, 'Disc+art': 2, 'Disc+Peek': 7, 'Discectomía': 11, 'Discectomía artrodesis.': 4, 'Discectomía DIE': 1, 'Discectomía doble y caja': 1, 'Discectomía+ foraminotomía': 1, 'Embolización': 1, 'Exéresis': 2, 'Fusión': 3, 'Fusión Lumbar': 2, 'Laminectomía': 11, 'Microdiscectomía': 1, 'P80.51': 6, 'Retirada tornillos': 3, 'Retirada tornillos ': 3, 'Terapia intratecal Baclofeno': 1}


In [29]:
model.const1= Constraint(expr= model.x['Abordaje anterior'] <= 2)
model.const2= Constraint(expr= model.x['Abordaje endoscópico'] <= 3)
model.const3= Constraint(expr= model.x['Artrodesis cervical'] <= 4)
model.const4= Constraint(expr= model.x['Artrodesis lumbar'] <= 4)
model.const5= Constraint(expr= model.x['Craneotomía'] <= 5)
model.const6= Constraint(expr= model.x['Disc c3-c4 yc 4-c5 + PEEK'] <= 1)
model.const7= Constraint(expr= model.x['Disc caja'] <= 2)
model.const8= Constraint(expr= model.x['disc caja placa'] <= 2)
model.const9= Constraint(expr= model.x['Disc+Peek'] <= 7)
model.const10= Constraint(expr= model.x['Disc+art'] <= 2)
model.const11= Constraint(expr= model.x['Discectomía'] <= 11)
model.const12= Constraint(expr= model.x['Discectomía artrodesis.'] <= 4)
model.const13= Constraint(expr= model.x['Discectomía DIE'] <= 2)
model.const14= Constraint(expr= model.x['Exéresis'] <= 2)
model.const15= Constraint(expr= model.x['Fusión'] <= 3)
model.const16= Constraint(expr= model.x['Fusión Lumbar'] <= 2)
model.const17= Constraint(expr= model.x['Laminectomía'] <= 11)
model.const18= Constraint(expr= model.x['P80.51'] <= 6)
model.const19= Constraint(expr= model.x['Retirada tornillos'] <= 3)
model.const20= Constraint(expr= model.x['Descompresión'] <= 2)
model.const21= Constraint(expr= model.x['Craneoplastia'] <= 1)
model.const22= Constraint(expr= model.x['Discectomía+ foraminotomía'] <= 1)
model.const23= Constraint(expr= model.x['Embolización'] <= 1)
model.const24= Constraint(expr= model.x['Microdiscectomía'] <= 1)
model.const25= Constraint(expr= model.x['Terapia intratecal Baclofeno'] <= 1)

To obtain the solution, I will use the glpk solver

In [30]:
opt = SolverFactory('glpk')

results = opt.solve(model) 
 
model.display()



Model unknown

  Variables:
    x : Size=27, Index=x_index
        Key                          : Lower : Value : Upper : Fixed : Stale : Domain
                   Abordaje anterior :     0 :   2.0 :  None : False : False : NonNegativeIntegers
                Abordaje endoscópico :     0 :   0.0 :  None : False : False : NonNegativeIntegers
                 Artrodesis cervical :     0 :   0.0 :  None : False : False : NonNegativeIntegers
                   Artrodesis lumbar :     0 :   3.0 :  None : False : False : NonNegativeIntegers
                       Craneoplastia :     0 :   0.0 :  None : False : False : NonNegativeIntegers
                         Craneotomía :     0 :   5.0 :  None : False : False : NonNegativeIntegers
                       Descompresión :     0 :   0.0 :  None : False : False : NonNegativeIntegers
           Disc c3-c4 yc 4-c5 + PEEK :     0 :   0.0 :  None : False : False : NonNegativeIntegers
                           Disc caja :     0 :   0.0 :  None : 

In [31]:
print("Total hours in a week=",model.objective())
print('\nSelected Procedures:')

for i in proc:
    if model.x[i]()>0:
        print('Procedure:', i, ', Number of surgeries:', model.x[i]())

Total hours in a week= 40.0

Selected Procedures:
Procedure: Artrodesis lumbar , Number of surgeries: 3.0
Procedure: Abordaje anterior , Number of surgeries: 2.0
Procedure: Craneotomía , Number of surgeries: 5.0


This **solution** shows that in order to maximize the hours operating in the first week, a doctor should operate 5 'Craneotomía' (22.5 hrs), 3 'Artrodesis lumbar' (10.5 hrs) and 2 'Abordaje anterior' (7 hrs)

## Exercise c: Sensitivities of each constraint


Since the solutions to my problem are integers, I cannot use the dual function to get the sensitivities, therefore I changed the constraints each by each to see how the solution changes.

· If I change the max number of hours from 40 to 41, the total hours stay being the maximum possible (41), but now I get some different procedures. Instead of getting 3 'Artrodesis lumbar', I get 1. In addition, I get a new procedure, 2 surgeries of 'Abordaje endoscópico'. If I keep increasing the maximum, it will obviously keep incresing the number of hours, since we are looking to optimize that. 

· If I change the minimum number of hours a doctor has to work in a week, the solution is not going to change. This is because I am trying to maximize the hours. However, I think I should keep this constraint just in case I want to change the problem and so it never gives a value of 0 procedures in th solution.

· If I add 1 to the constraint of 'Abordaje anterior', the solution also changes. The doctor should operate now 3 'Abordaje anterior' instead of 2. If I keep increasing that value, the same thing happens to the solution. If I substract 1 instead of adding, the solution also changes. Now, it takes another procedure because there are not enough 'Abordaje anterior' to fill the hours.

· Basically, it happens the same thing with the other constraints. Although, there are many procedures that the program is not selecting, these will be important for the future. When this first week ends, the procedures that were completed, will be erased from the list by the doctors, so all the constraints have the same level of importance.

## Exercise d: Modify the problem

In this problem, I want to see which patients will be able to have surgery the first week.

· **Decision variables:** the different procedures, (I want to see if an operation is done or not)

· **Objective:** maximize surgery hours

· **Constraints:** max hours per week, min hours per week, compatibility between procedures 

In this model, I am taking into account that there can only be 1 operation per procedure each week.

In [32]:
model2= ConcreteModel()

#Define variables

model2.x=Var(proc, domain=Binary)

#Objective function: we must determine which patients to operate taking into account we want to maximize the hours
def obj_expression(model):
    return sum(model2.x[i] * hours[i] for i in proc)

model2.objective=Objective(rule=obj_expression, sense=maximize)

In [33]:
#constraints
def upper_limit(model):
    suma= sum(model2.x[i] * hours[i] for i in proc) 
    return suma <= 40

def lower_limit(model):
    suma= sum(model2.x[i] * hours[i] for i in proc) 
    return suma >= 1 

model2.constraint1=Constraint(rule=upper_limit)
model2.constraint2=Constraint(rule=lower_limit)

There are some procedures that need to be done the same week because the doctors use the same materials. There are also some procedures that are not compatible in the same week. For instance, the procedures 'Abordaje endoscópico' and 'Craneotomía' cannot be done in the same week, or 'Abordaje anterior' and 'Disc+Peek' must be done in the same week.

In [34]:
model2.const1= Constraint(expr= model2.x['Abordaje endoscópico'] +  model2.x['Craneotomía'] <= 1)
model2.const2= Constraint(expr= model2.x['Abordaje anterior'] - model2.x['Disc+Peek'] <= 0)
model2.const3= Constraint(expr= model2.x['Artrodesis cervical'] + model2.x['Artrodesis lumbar']  <= 1)
model2.const4= Constraint(expr= model2.x['Disc c3-c4 yc 4-c5 + PEEK'] + model2.x['disc caja placa'] <= 1)
model2.const5= Constraint(expr= model2.x['Disc caja'] - model2.x['Laminectomía'] <= 0)

In [35]:
opt = SolverFactory('glpk')

results = opt.solve(model2)  # solve the model with the selected solver
 
model2.display()

Model unknown

  Variables:
    x : Size=27, Index=x_index
        Key                          : Lower : Value : Upper : Fixed : Stale : Domain
                   Abordaje anterior :     0 :   1.0 :     1 : False : False : Binary
                Abordaje endoscópico :     0 :   0.0 :     1 : False : False : Binary
                 Artrodesis cervical :     0 :   0.0 :     1 : False : False : Binary
                   Artrodesis lumbar :     0 :   1.0 :     1 : False : False : Binary
                       Craneoplastia :     0 :   1.0 :     1 : False : False : Binary
                         Craneotomía :     0 :   1.0 :     1 : False : False : Binary
                       Descompresión :     0 :   1.0 :     1 : False : False : Binary
           Disc c3-c4 yc 4-c5 + PEEK :     0 :   0.0 :     1 : False : False : Binary
                           Disc caja :     0 :   1.0 :     1 : False : False : Binary
                           Disc+Peek :     0 :   1.0 :     1 : False : False : Bi

In [36]:
print("Total hours in a week=",model2.objective())
print('\nPatients getting surgery:')

for i in proc:
    if model2.x[i]()==1:
        print('Patient:', patient[i])

Total hours in a week= 40.0

Patients getting surgery:
Patient: 5
Patient: 78
Patient: 54
Patient: 71
Patient: 87
Patient: 91
Patient: 19
Patient: 68
Patient: 57
Patient: 81
Patient: 77
Patient: 88
Patient: 25
Patient: 58
Patient: 80
Patient: 89
Patient: 84
Patient: 11
Patient: 65
Patient: 15
Patient: 75
Patient: 28
Patient: 22


Above you can see the patients that will get their surgery the first week because their procedures meet all the constraints.